In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,RidgeCV,LassoCV,ElasticNetCV
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
import pickle

In [2]:
df=pd.read_csv("../data/yield_prediction.csv")


In [3]:
df

,Unnamed: 0,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,production,Yield
0,125191,Maharashtra,AHMEDNAGAR,1997,Autumn,Maize,1.0,20.770894,2.068260,1014.863796,21.947147,loamy,56.070,0.000,0.000,1113.0,1113.000000
1,125192,Maharashtra,AHMEDNAGAR,1997,Kharif,Arhar/Tur,17600.0,20.160426,1.976480,1015.193528,20.643237,sandy,9.000,9.000,0.000,6300.0,0.357955
2,125193,Maharashtra,AHMEDNAGAR,1997,Kharif,Bajra,274100.0,21.998299,2.000524,1014.184607,21.422312,clay,0.000,0.000,0.000,152800.0,0.557461
3,125194,Maharashtra,AHMEDNAGAR,1997,Kharif,Gram,40800.0,21.776377,2.019750,1015.053118,21.810567,chalky,38.250,38.250,38.250,18600.0,0.455882
4,125195,Maharashtra,AHMEDNAGAR,1997,Kharif,Jowar,900.0,20.075734,1.974351,1015.170238,21.930206,clay,0.000,23.184,0.000,1100.0,1.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12623,137814,Maharashtra,YAVATMAL,2014,Rabi,Jowar,4000.0,21.635879,2.000060,1014.302213,20.060662,silty,7.840,7.840,0.000,3300.0,0.825000
12624,137815,Maharashtra,YAVATMAL,2014,Rabi,Maize,1300.0,21.709611,2.053609,1015.803912,21.263478,silt,7.504,18.760,7.504,200.0,0.153846
12625,137816,Maharashtra,YAVATMAL,2014,Rabi,Wheat,29100.0,21.851730,2.027476,1014.031903,20.059945,sandy,0.000,0.000,0.000,26800.0,0.920962
12626,137817,Maharashtra,YAVATMAL,2014,Summer,Groundnut,9400.0,21.569380,2.004421,1013.989125,21.835158,chalky,2.896,3.620,0.000,11500.0,1.223404


In [4]:
df['soil_type'].unique()

array(['loamy', 'sandy', 'clay', 'chalky', 'peaty', 'silty', 'silt'],
      dtype=object)

In [5]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [6]:
df.isnull().sum()


state_names         0
district_names      0
crop_year           0
season_names        0
crop_names          0
area                0
temperature         0
wind_speed          0
pressure            0
humidity            0
soil_type           0
N                   0
P                   0
K                   0
production        132
Yield             132
dtype: int64

In [13]:
#df['production']=df['production'].fillna(df['production'].mean())
#df['Yield']=df['Yield'].fillna(df['Yield'].mean())

In [7]:
df=df[df['crop_year']>=2004]
df

,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,production,Yield
212,Maharashtra,AHMEDNAGAR,2004,Kharif,Arhar/Tur,12200.0,20.768143,2.002031,1013.280471,20.427922,peaty,10.500,27.300,27.300,4800.0,0.393443
213,Maharashtra,AHMEDNAGAR,2004,Kharif,Bajra,240500.0,20.722713,2.105239,1015.061641,20.468584,silty,39.720,105.920,52.960,143400.0,0.596258
214,Maharashtra,AHMEDNAGAR,2004,Kharif,Groundnut,5300.0,21.419190,2.046843,1015.770055,21.836158,peaty,8.008,8.008,0.000,3900.0,0.735849
215,Maharashtra,AHMEDNAGAR,2004,Kharif,Jowar,100.0,20.425919,2.024060,1013.971163,21.028403,sandy,5.824,14.560,5.824,100.0,1.000000
216,Maharashtra,AHMEDNAGAR,2004,Kharif,Maize,11400.0,20.823344,1.989898,1015.453191,20.340815,silty,0.000,0.000,0.000,14000.0,1.228070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12623,Maharashtra,YAVATMAL,2014,Rabi,Jowar,4000.0,21.635879,2.000060,1014.302213,20.060662,silty,7.840,7.840,0.000,3300.0,0.825000
12624,Maharashtra,YAVATMAL,2014,Rabi,Maize,1300.0,21.709611,2.053609,1015.803912,21.263478,silt,7.504,18.760,7.504,200.0,0.153846
12625,Maharashtra,YAVATMAL,2014,Rabi,Wheat,29100.0,21.851730,2.027476,1014.031903,20.059945,sandy,0.000,0.000,0.000,26800.0,0.920962
12626,Maharashtra,YAVATMAL,2014,Summer,Groundnut,9400.0,21.569380,2.004421,1013.989125,21.835158,chalky,2.896,3.620,0.000,11500.0,1.223404


In [12]:
df.columns.shape

(16,)

In [10]:
df.to_csv('newy.csv')

In [8]:
df['season_names'].unique()

array(['Kharif     ', 'Rabi       ', 'Summer     ', 'Whole Year '],
      dtype=object)

In [15]:
categorical_feature=[feature for feature in df.columns if df[feature].dtypes=='O']

categorical_feature

['state_names', 'district_names', 'season_names', 'crop_names', 'soil_type']

In [16]:
for feature in categorical_feature:
    labels_ordered=df.groupby([feature])['Yield'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df[feature]=df[feature].map(labels_ordered)

C:\Users\Dell\AppData\Local\Temp\ipykernel_16308\1593447033.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature]=df[feature].map(labels_ordered)
C:\Users\Dell\AppData\Local\Temp\ipykernel_16308\1593447033.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature]=df[feature].map(labels_ordered)
C:\Users\Dell\AppData\Local\Temp\ipykernel_16308\1593447033.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [17]:
df

,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,production,Yield
212,0,18,2004,2,13,12200.0,20.768143,2.002031,1013.280471,20.427922,3,10.500,27.300,27.300,4800.0,0.393443
213,0,18,2004,2,14,240500.0,20.722713,2.105239,1015.061641,20.468584,0,39.720,105.920,52.960,143400.0,0.596258
214,0,18,2004,2,18,5300.0,21.419190,2.046843,1015.770055,21.836158,3,8.008,8.008,0.000,3900.0,0.735849
215,0,18,2004,2,16,100.0,20.425919,2.024060,1013.971163,21.028403,1,5.824,14.560,5.824,100.0,1.000000
216,0,18,2004,2,24,11400.0,20.823344,1.989898,1015.453191,20.340815,0,0.000,0.000,0.000,14000.0,1.228070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12623,0,30,2014,0,16,4000.0,21.635879,2.000060,1014.302213,20.060662,0,7.840,7.840,0.000,3300.0,0.825000
12624,0,30,2014,0,24,1300.0,21.709611,2.053609,1015.803912,21.263478,6,7.504,18.760,7.504,200.0,0.153846
12625,0,30,2014,0,22,29100.0,21.851730,2.027476,1014.031903,20.059945,1,0.000,0.000,0.000,26800.0,0.920962
12626,0,30,2014,1,18,9400.0,21.569380,2.004421,1013.989125,21.835158,2,2.896,3.620,0.000,11500.0,1.223404


In [18]:
df['Yield']=df['production']/df['area']

C:\Users\Dell\AppData\Local\Temp\ipykernel_16308\2490600354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Yield']=df['production']/df['area']


In [19]:
df.drop(['production'],axis=1)

,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,Yield
212,0,18,2004,2,13,12200.0,20.768143,2.002031,1013.280471,20.427922,3,10.500,27.300,27.300,0.393443
213,0,18,2004,2,14,240500.0,20.722713,2.105239,1015.061641,20.468584,0,39.720,105.920,52.960,0.596258
214,0,18,2004,2,18,5300.0,21.419190,2.046843,1015.770055,21.836158,3,8.008,8.008,0.000,0.735849
215,0,18,2004,2,16,100.0,20.425919,2.024060,1013.971163,21.028403,1,5.824,14.560,5.824,1.000000
216,0,18,2004,2,24,11400.0,20.823344,1.989898,1015.453191,20.340815,0,0.000,0.000,0.000,1.228070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12623,0,30,2014,0,16,4000.0,21.635879,2.000060,1014.302213,20.060662,0,7.840,7.840,0.000,0.825000
12624,0,30,2014,0,24,1300.0,21.709611,2.053609,1015.803912,21.263478,6,7.504,18.760,7.504,0.153846
12625,0,30,2014,0,22,29100.0,21.851730,2.027476,1014.031903,20.059945,1,0.000,0.000,0.000,0.920962
12626,0,30,2014,1,18,9400.0,21.569380,2.004421,1013.989125,21.835158,2,2.896,3.620,0.000,1.223404


In [20]:
df

,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,production,Yield
212,0,18,2004,2,13,12200.0,20.768143,2.002031,1013.280471,20.427922,3,10.500,27.300,27.300,4800.0,0.393443
213,0,18,2004,2,14,240500.0,20.722713,2.105239,1015.061641,20.468584,0,39.720,105.920,52.960,143400.0,0.596258
214,0,18,2004,2,18,5300.0,21.419190,2.046843,1015.770055,21.836158,3,8.008,8.008,0.000,3900.0,0.735849
215,0,18,2004,2,16,100.0,20.425919,2.024060,1013.971163,21.028403,1,5.824,14.560,5.824,100.0,1.000000
216,0,18,2004,2,24,11400.0,20.823344,1.989898,1015.453191,20.340815,0,0.000,0.000,0.000,14000.0,1.228070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12623,0,30,2014,0,16,4000.0,21.635879,2.000060,1014.302213,20.060662,0,7.840,7.840,0.000,3300.0,0.825000
12624,0,30,2014,0,24,1300.0,21.709611,2.053609,1015.803912,21.263478,6,7.504,18.760,7.504,200.0,0.153846
12625,0,30,2014,0,22,29100.0,21.851730,2.027476,1014.031903,20.059945,1,0.000,0.000,0.000,26800.0,0.920962
12626,0,30,2014,1,18,9400.0,21.569380,2.004421,1013.989125,21.835158,2,2.896,3.620,0.000,11500.0,1.223404


In [21]:
import scipy.stats as stat

In [22]:
df['area'],parameters=stat.boxcox(df['area']+1)
df['N'],parameters=stat.boxcox(df['N']+1)
df['P'],parameters=stat.boxcox(df['P']+1)
df['K'],parameters=stat.boxcox(df['K']+1)

C:\Users\Dell\AppData\Local\Temp\ipykernel_16308\828050052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['area'],parameters=stat.boxcox(df['area']+1)
C:\Users\Dell\AppData\Local\Temp\ipykernel_16308\828050052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['N'],parameters=stat.boxcox(df['N']+1)
C:\Users\Dell\AppData\Local\Temp\ipykernel_16308\828050052.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [23]:
df.head()

,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,production,Yield
212,0,18,2004,2,13,12.518226,20.768143,2.002031,1013.280471,20.427922,3,1.815898,2.566763,0.848913,4800.0,0.393443
213,0,18,2004,2,14,18.137950,20.722713,2.105239,1015.061641,20.468584,0,2.393929,3.252428,0.858557,143400.0,0.596258
214,0,18,2004,2,18,11.113124,21.419190,2.046843,1015.770055,21.836158,3,1.681027,1.842314,0.000000,3900.0,0.735849
215,0,18,2004,2,16,5.292411,20.425919,2.024060,1013.971163,21.028403,1,1.517092,2.205374,0.772555,100.0,1.000000
216,0,18,2004,2,24,12.401360,20.823344,1.989898,1015.453191,20.340815,0,0.000000,0.000000,0.000000,14000.0,1.228070


In [24]:
feature_scale=[feature for feature in df.columns if feature not in ['Unnamed: 0','Yield']]
from sklearn.preprocessing import MinMaxScaler
min_max=MinMaxScaler()
min_max=min_max.fit_transform(df[feature_scale])
df_minmax=pd.DataFrame(min_max,columns=feature_scale)
df_minmax.head()

,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,production
0,0.0,0.545455,0.0,0.666667,0.52,0.598681,0.363026,0.228776,0.093378,0.197983,0.500000,0.562679,0.783848,0.988228,0.000239
1,0.0,0.545455,0.0,0.666667,0.56,0.883537,0.340963,0.966317,0.687203,0.217723,0.000000,0.741789,0.993239,0.999455,0.007152
2,0.0,0.545455,0.0,0.666667,0.72,0.527458,0.679207,0.549013,0.923382,0.881633,0.500000,0.520887,0.562613,0.000000,0.000195
3,0.0,0.545455,0.0,0.666667,0.64,0.232414,0.196825,0.386199,0.323648,0.489495,0.166667,0.470090,0.673486,0.899340,0.000005
4,0.0,0.545455,0.0,0.666667,0.96,0.592757,0.389834,0.142071,0.817742,0.155696,0.000000,0.000000,0.000000,0.000000,0.000698


In [25]:
data = pd.concat([df[['Yield']].reset_index(drop=True),pd.DataFrame(df_minmax,columns=feature_scale)],axis=1)

In [26]:
data.head()

,Yield,state_names,district_names,crop_year,season_names,crop_names,area,temperature,wind_speed,pressure,humidity,soil_type,N,P,K,production
0,0.393443,0.0,0.545455,0.0,0.666667,0.52,0.598681,0.363026,0.228776,0.093378,0.197983,0.500000,0.562679,0.783848,0.988228,0.000239
1,0.596258,0.0,0.545455,0.0,0.666667,0.56,0.883537,0.340963,0.966317,0.687203,0.217723,0.000000,0.741789,0.993239,0.999455,0.007152
2,0.735849,0.0,0.545455,0.0,0.666667,0.72,0.527458,0.679207,0.549013,0.923382,0.881633,0.500000,0.520887,0.562613,0.000000,0.000195
3,1.000000,0.0,0.545455,0.0,0.666667,0.64,0.232414,0.196825,0.386199,0.323648,0.489495,0.166667,0.470090,0.673486,0.899340,0.000005
4,1.228070,0.0,0.545455,0.0,0.666667,0.96,0.592757,0.389834,0.142071,0.817742,0.155696,0.000000,0.000000,0.000000,0.000000,0.000698


In [27]:
data=data.fillna(data.mean())

In [28]:
x1=data.drop(['Yield'],axis=1)
y1=data['Yield']

In [29]:
len(x1.columns)

15

In [30]:
acc=[]
modelname=[]

In [31]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x1,y1,test_size = 0.20,random_state =10)

In [32]:
from sklearn.metrics import r2_score
lr=LinearRegression()
lr.fit(Xtrain,Ytrain)
predicted_values1=lr.predict(Xtest)
r2_score(Ytest,predicted_values1)

0.4674305895070183

In [33]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(Xtrain,Ytrain)
predicted_values2 = regressor.predict(Xtest)
r2_score(Ytest,predicted_values2)

0.9382599074728837

In [34]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=30,random_state=0)
rf.fit(Xtrain, Ytrain)
predicted_values3=rf.predict(Xtest)
r2_score(Ytest,predicted_values3)

0.9675161278738303

In [41]:
import pickle
# Dump the trained DecisionTree classifier with Pickle
DT_pkl_filename = '../models/RandomForestRegressor.pkl'
# Open the file to save as pkl file
DT_Model_pkl = open(DT_pkl_filename, 'wb')
pickle.dump(rf, DT_Model_pkl)
# Close the pickle instances
DT_Model_pkl.close()

In [36]:
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf')
svr.fit(Xtrain, Ytrain)

SVR()

In [37]:

y_pred = regressor.predict(Xtest)

In [38]:
r2_score(Ytest,y_pred)

0.9382599074728837

In [39]:
import pickle
# Dump the trained DecisionTree classifier with Pickle
DT_pkl_filename = '../models/CYFSupportVectorRegression.pkl'
# Open the file to save as pkl file
DT_Model_pkl = open(DT_pkl_filename, 'wb')
pickle.dump(svr, DT_Model_pkl)
# Close the pickle instances
DT_Model_pkl.close()

In [40]:
data = np.array([[63.0,	42,	21.0,	23.262376,	72.331255,	5.798424,	67.102251	]])
prediction = svc1.predict(data)
print(prediction)

NameError: name 'svc1' is not defined